In [1]:
import pandas as pd 
import sqlite3 

In [2]:
conn = sqlite3.connect('/kaggle/input/chinook-sqlite/Chinook_Sqlite.sqlite')
c = conn.cursor()

In [3]:
def sq(str,con=conn):
    return pd.read_sql('''{}'''.format(str), con)

#### 1. Write a query that returns each genre, with the number of tracks sold in the USA:
*    in absolute numbers
*    in percentages

In [4]:
sq('''SELECT g.name,
        c.Country,
        COUNT(il.InvoiceLineId) AS NumOfTracksSold,
        (CAST(COUNT(il.InvoiceLineId) AS FLOAT)/
        (SELECT (COUNT(i.InvoiceId))
         FROM Invoice
         INNER JOIN
         InvoiceLine il ON t.TrackId = il.TrackId
         INNER JOIN
         Invoice i ON i.InvoiceId = il.InvoiceId
         WHERE c.Country = "USA")) AS PercentOfTracksSold

FROM Genre g
       INNER JOIN
       Track t ON g.GenreId = t.GenreId
       INNER JOIN
       InvoiceLine il ON t.TrackId = il.TrackId
       INNER JOIN
       Invoice i ON i.InvoiceId = il.InvoiceId
       INNER JOIN
       Customer c ON c.CustomerId = i.CustomerId

WHERE 
    c.Country = "USA"
GROUP BY 1
ORDER BY NumOfTracksSold DESC
LIMIT 10;
''')

,Name,Country,NumOfTracksSold,PercentOfTracksSold
0,Rock,USA,157,0.381068
1,Latin,USA,91,0.110437
2,Metal,USA,64,0.155340
3,Alternative & Punk,USA,50,0.121359
4,Jazz,USA,22,0.053398
5,Blues,USA,15,0.018204
6,TV Shows,USA,14,0.033981
7,R&B/Soul,USA,12,0.014563
8,Comedy,USA,8,0.019417
9,Classical,USA,8,0.019417


#### 1. Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company. Add any extra attributes for that employee that you find are relevant to the analysis.

In [5]:
sq('''WITH Support_Agent AS
(SELECT
    EmployeeId AS SupportAgentId,
    (e.FirstName || " " || e.LastName) AS Name,
    SUM(i.Total) AS Total
    
 FROM
     Employee e
    JOIN Customer c ON c.SupportRepId = e.EmployeeId
    JOIN Invoice i ON i.CustomerId = c.CustomerId
    
 WHERE
     EmployeeId IN (3, 4, 5)
 GROUP BY 
     Name, c.CustomerId
)

SELECT
    sa.Name,
    SUM(i.Total) AS Total

FROM 
    Support_Agent sa
    JOIN Customer c ON c.CustomerId = sa.SupportAgentId
    JOIN Invoice i ON i.CustomerId = c.CustomerId
        
GROUP BY Name
''')

,Name,Total
0,Jane Peacock,832.02
1,Margaret Park,792.40
2,Steve Johnson,731.16


#### 1. Write a query that collates data on purchases from different countries.
   *  Where a country has only one customer, collect them into an "Other" group.
   *  The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.
   
#### 2. For each country, include:
   *  total number of customers
   *  total value of sales
   *  average value of sales per customer
   *  average order value

In [6]:
sq('''WITH OtherCountry AS
(SELECT
    CASE
        WHEN (SELECT  COUNT(country)
                 FROM Customer
                 WHERE Country = c.Country) = 1  THEN 'Other'
         ELSE c.Country
     END AS Country, 
     c.CustomerId,
     il.*

 FROM Customer c
 JOIN Invoice i ON c.CustomerId = i.CustomerId
 JOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId
)

SELECT
    Country,
    TotalCustomer,
    TotalSales,
    AvgSalesPerCustomer,
    AvgOrderValue

FROM
    (SELECT 
        Country,
        COUNT(CustomerId) AS TotalCustomer,
        CASE
            WHEN Country = 'Other' THEN 1
            ELSE 0
        END AS sort,
        ROUND(SUM(UnitPrice), 2) AS TotalSales,
        ROUND(SUM(UnitPrice)/COUNT(DISTINCT CustomerId), 2) AS AvgSalesPerCustomer,
        ROUND(SUM(UnitPrice)/Count(InvoiceLineId), 2) AS AvgOrderValue
     
     FROM OtherCountry
     
     GROUP BY Country
     ORDER BY sort, TotalSales DESC
    )
''')

,Country,TotalCustomer,TotalSales,AvgSalesPerCustomer,AvgOrderValue
0,USA,494,523.06,40.24,1.06
1,Canada,304,303.96,38.00,1.00
2,France,190,195.10,39.02,1.03
3,Brazil,190,190.10,38.02,1.00
4,Germany,152,156.48,39.12,1.03
5,United Kingdom,114,112.86,37.62,0.99
6,Czech Republic,76,90.24,45.12,1.19
7,Portugal,76,77.24,38.62,1.02
8,India,74,75.26,37.63,1.02
9,Other,570,604.30,40.29,1.06
